In [ ]:
#FAIL - gets incorrect mouse locations on screen - use manual MPos app instead :(

#CONFIG ONLY - RUN ONCE PER CHART

import tkinter as tk

def get_region(title="Select Region"):
    root = tk.Tk()
    root.attributes('-alpha', 0.3)  # semi-transparent
    root.attributes('-topmost', True)
    root.title(title)
    screen_width = root.winfo_screenwidth()
    screen_height = root.winfo_screenheight()
    canvas = tk.Canvas(root, width=screen_width, height=screen_height)
    canvas.pack()
    region = {}

    def on_mouse_down(event):
        region['x1'] = event.x
        region['y1'] = event.y
        region['rect'] = canvas.create_rectangle(event.x, event.y, event.x, event.y, outline='red')

    def on_mouse_drag(event):
        if 'rect' in region:
            canvas.coords(region['rect'], region['x1'], region['y1'], event.x, event.y)

    def on_mouse_up(event):
        region['x2'] = event.x
        region['y2'] = event.y
        root.destroy()

    canvas.bind("<Button-1>", on_mouse_down)
    canvas.bind("<B1-Motion>", on_mouse_drag)
    canvas.bind("<ButtonRelease-1>", on_mouse_up)
    root.mainloop()

    x1, y1 = min(region['x1'], region['x2']), min(region['y1'], region['y2'])
    x2, y2 = max(region['x1'], region['x2']), max(region['y1'], region['y2'])
    w, h = x2 - x1, y2 - y1
    return (x1, y1, w, h)

initdelay=3.5
print(f'starting in {initdelay}s, open barchart screen fully')
print("Drag to select CHART region, then release mouse.")
chart_region = get_region("Select CHART region")
print("Drag to select TOOLTIP region, then release mouse.")
tooltip_region = get_region("Select TOOLTIP region")
print("Drag to select VOLUME region, then release mouse.")
volume_region = get_region("Select VOLUME region")
print("Drag to select DATE region, then release mouse.")
date_region = get_region("Select DATE region")

print("Regions:")
print("CHART_REGION =", chart_region)
print("TOOLTIP_REGION =", tooltip_region)
print("VOLUME_REGION =", volume_region)
print("DATE_region =",date_region)


starting in 3.5s, open barchart screen fully
Drag to select CHART region, then release mouse.
Drag to select TOOLTIP region, then release mouse.
Drag to select VOLUME region, then release mouse.
Drag to select DATE region, then release mouse.
Regions:
CHART_REGION = (97, 137, 1356, 447)
TOOLTIP_REGION = (304, 98, 242, 22)
VOLUME_REGION = (207, 613, 87, 18)
DATE_region = (39, 99, 82, 22)


In [ ]:
import pyautogui
import pytesseract
#from PIL import Image
import time
import pandas as pd
#import cv2
#import numpy as np
import keyboard
import re
# ---- CONFIG ---- #
# Set these manually for your setup
#FULLSCREEN WINDOW open, fullscreened

CHART_TOP_LEFT = (chart_region[0],chart_region[1])#(117, 338)   # x, y of chart's top-left corner
CHART_BOTTOM_RIGHT = (chart_region[0]+chart_region[2],chart_region[1]+chart_region[3])  # x, y of chart's bottom-right
TOOLTIP_REGION = tooltip_region#(392, 164, 300, 25)
VOLUME_REGION = volume_region#(270, 803, 100, 25)
DATE_REGION = date_region#(58, 164, 150, 28)  # example: x, y, w, h for date
HORIZONTAL_STEPS =CHART_BOTTOM_RIGHT[0]-CHART_TOP_LEFT[0] -5 #cushion
STEP_SIZE = 1 #pixel each 
#no formula needed lol^#int((CHART_BOTTOM_RIGHT[0] - CHART_TOP_LEFT[0]) / HORIZONTAL_STEPS)
DELAY = 0 #TODO change faster
initdelay = 3.5 #setup s
data = []
seen_dates = set()

print(f'starting, u have {initdelay}s')
time.sleep(initdelay)

for i in range(HORIZONTAL_STEPS):
    if keyboard.is_pressed('q'):
        print("Quit key pressed. Exiting loop.")
        break
    while keyboard.is_pressed('p'):
        print("Paused. Press 'r' to resume or 'q' to quit.")
        while not keyboard.is_pressed('r'):
            if keyboard.is_pressed('q'):
                print("Quit key pressed. Exiting loop.")
                exit()
            time.sleep(0.1)
        print("Resuming.")
        time.sleep(0.2)  # Avoid accidental double-trigger

    x = CHART_TOP_LEFT[0] + i * STEP_SIZE
    y = int((CHART_TOP_LEFT[1] + CHART_BOTTOM_RIGHT[1]) / 2)
    pyautogui.moveTo(x, y)
    time.sleep(DELAY)

    # ---- Date OCR ----
    date_img = pyautogui.screenshot(region=DATE_REGION)
    text = pytesseract.image_to_string(
    date_img.convert('L'), config='--psm 7 -c tessedit_char_whitelist=0123456789.,'
).strip()
    print(f"dateOCR: '{text}'")
    date = re.search(r'\d{8}',text)
    if date:
        date = date.group()
        date = f"{date[:2]}/{date[2:4]}/{date[4:]}"
    else:
        date = None  # or set to date_text for debugging

    # ---- Tooltip OCR (OHLC) ----
    tooltip_img = pyautogui.screenshot(region=TOOLTIP_REGION)
    text = pytesseract.image_to_string(
        tooltip_img.convert('L'),
        config='--psm 6 -c tessedit_char_whitelist=OHLC0123456789. '
    ).strip()
    print(f"OCR OHLC text: '{text}'")  # Debug

    # 1. Try to extract values by label
    matches = re.findall(r'([OHLC])\s*([0-9]*\.[0-9]+)', text)
    d = {k.lower(): v for k, v in matches}

    # 2. Fallback: If missing any of o/h/l/c, extract all numbers in order
    nums = re.findall(r'[0-9]*\.[0-9]+', text)
    if len(nums) == 4:
        # Always assign by standard order
        ohlc_keys = ['o', 'h', 'l', 'c']
        for k, v in zip(ohlc_keys, nums):
            d.setdefault(k, v)  # Don't overwrite existing matches
    elif len(nums) == 3:
        # Sometimes Open gets mangled, try best-effort mapping
        ohlc_keys = ['h', 'l', 'c'] if 'o' not in d else ['o', 'h', 'c']
        for k, v in zip(ohlc_keys, nums):
            d.setdefault(k, v)

    print(f"Extracted OHLC: {d}")

    # ---- Volume OCR ----
    vol_img = pyautogui.screenshot(region=VOLUME_REGION)
    vol_text = pytesseract.image_to_string(vol_img, config='--psm 7 -c tessedit_char_whitelist=0123456789,').strip()
    #print(f"OCR result: '{vol_text}'")

    vol_val = None
    for word in vol_text.split():
        if word.replace(',', '').isdigit():
            vol_val = word.replace(',', '')
            break
    #print(f"OCR volume region raw output: '{vol_text}'")
    #vol_img = pyautogui.screenshot(region=VOLUME_REGION)
    #vol_img.save(f'vol_test_{i}.png')#q

    if date and date not in seen_dates:
        outrow = {'date': date}
        outrow.update(d)
        outrow['volume'] = vol_val
        data.append(outrow)
        seen_dates.add(date)
        print(f"{i+1}/{HORIZONTAL_STEPS}|{date or 'No date'} | {outrow}")
    else:
        
        print(f"{i+1}/{HORIZONTAL_STEPS}|{date or 'No date'} | (duplicate or invalid, skipped)")
# ---- Output ----
df = pd.DataFrame(data)
df.to_csv('mics_chart_ohlcv.csv', index=False)
print("Saved to mics_chart_ohlcv.csv")

starting, u have 3.5s
dateOCR: ''
OCR OHLC text: ''
Extracted OHLC: {}
1/1351|No date | (duplicate or invalid, skipped)
dateOCR: ''
OCR OHLC text: ''
Extracted OHLC: {}
2/1351|No date | (duplicate or invalid, skipped)
dateOCR: ''
OCR OHLC text: ''
Extracted OHLC: {}
3/1351|No date | (duplicate or invalid, skipped)
dateOCR: ''
OCR OHLC text: ''
Extracted OHLC: {}
4/1351|No date | (duplicate or invalid, skipped)
dateOCR: ''
OCR OHLC text: ''
Extracted OHLC: {}
5/1351|No date | (duplicate or invalid, skipped)
dateOCR: ''
OCR OHLC text: ''
Extracted OHLC: {}
6/1351|No date | (duplicate or invalid, skipped)
dateOCR: ''
OCR OHLC text: ''
Extracted OHLC: {}
7/1351|No date | (duplicate or invalid, skipped)
dateOCR: ''
OCR OHLC text: ''
Extracted OHLC: {}
8/1351|No date | (duplicate or invalid, skipped)
dateOCR: ''
OCR OHLC text: ''
Extracted OHLC: {}
9/1351|No date | (duplicate or invalid, skipped)
dateOCR: ''
OCR OHLC text: ''
Extracted OHLC: {}
10/1351|No date | (duplicate or invalid, skippe

KeyboardInterrupt: 

: 